# A brief summary of *Ethernet*-based *Networking*
<br>
<div style="opacity: 0.8; font-family: Consolas, Monaco, Lucida Console, Liberation Mono, DejaVu Sans Mono, Bitstream Vera Sans Mono, Courier New; font-size: 12px; font-style: italic;">
    ────────
    for more from the author, visit
    <a href="https://github.com/hazemanwer2000">github.com/hazemanwer2000</a>.
    ────────
</div>

## Table of Contents
* [*Ethernet* LANs](#ethernet-lans)
  * [*Virtual LANs (VLANs)*](#virtual-lans-vlans)
  * [*Spanning Tree Protocol (STP)*](#spanning-tree-protocol-stp)


This document assumes some previous knowledge about *Networking*, including *OSI model*.

## *Ethernet* LANs <a class="anchor" id="ethernet-lans"></a>

An *Ethernet-based* LAN (i.e, *Local Area Network*) uses Ethernet frames to encapsulate and transport data from/to nodes within the network.

It is usually composed of a number of end-point devices, connected to a number of switches, as shown below.

<img src="../../.img/unannotated-lan.png" width="500" />

Every switch maintains a *MAC address table*.

Each entry in the table denotes a *MAC address*, the physical interface it is accessible through, and the *Virtual LAN* it resides within (see below).

Each entry in the table may be have been learned,
* statically (i.e, configured manually) or,
* dynamically (see below).

When an Ethernet frame is received on one of the physical interfaces of a switch,
1. the *source MAC address* is used to create a *dynamic* entry in the MAC address table,
2. the *destination MAC address* is used to decide, through which physical interface the Ethernet frame shall be forwarded.
    * If no matching entry was found in the MAC address table, the frame is forwarded out of all interfaces. This is known as a *Unicast Unknown Frame*.
    * In the case of a broadcast frame (i.e, *destination MAC address* of `FF:FF:FF:FF:FF:FF`), the frame is forwarded out of all interfaces, regardless. This is known as a *Broadcast Frame*.

### *Virtual LANs (VLANs)* <a class="anchor" id="virtual-lans-vlans"></a>

A physical *LAN* may be subdivided into a number of *Virtual LANs (VLANs)*.

For example, the figure below shows a physical *LAN*, composed of two *VLANs*.

Each physical interface on `SW1` has been configured, to be associated with a specific *VLAN*.

* [`CASE`]: `Dino` sends a broadcast frame. `SW1` receives the frame through a physical interface associated with `VLAN 1`. Hence, `SW1` forwards the frame out of all physical interfaces associated with `VLAN 1` (except `Dino`'s).

* [`CASE`]: `Dino` sends a unicast frame, destined to `Wilma`. `SW1` receives the frame through a physical interface associated with `VLAN 1`. Hence, `SW1` searches for a matching entry in its *MAC* address table, filtering out entries that do not belong to `VLAN 1`. Because no matching entry can be found, `SW1` forwards the frame out of all physical interfaces associated with `VLAN 1` (except `Dino`'s).

<img src="../../.img/2-virtual-lans.png" width="550" />

*Note:* To transport data across different *(V)LANs*, a *router* (i.e, a *Layer-3* networking device) is used (see below).

A physical interface between two switches must be *trunking*, if each switch is connected to more than one *VLAN*.

Over a *trunking* link, Ethernet frames are *VLAN-tagged*.

<img src="../../.img/vlan-trunking.png" width="450" />

* [`CASE`]: A broadcast frame is recieved over `Fa0/12`. `SW1` receives the frame through a physical interface associated with `VLAN 1`. Hence, `SW1` forwards the frame out of all physical interfaces associated with `VLAN 1` (except `Dino`'s), as well as any *trunking* interfaces. When `SW2` receives the broadcast frame over `Gi0/2`, it strips the *VLAN tag*, before forwarding the frames over `Fa0/22` and `Fa0/21`.

* [`CASE`]: A unicast frame is recieved over `Fa0/12`, destined to the end-point device connected `Fa0/21` on `SW2`. `SW1` receives the frame through a physical interface associated with `VLAN 1`. Hence, `SW1` searches for a matching entry in its *MAC* address table, filtering out entries that do not belong to `VLAN 1`. A matching entry is found, associated with `Gi0/1`. `SW1` forwards the *VLAN-tagged* frame over, and `SW2` strips the *VLAN tag*, before forwarding the frame over `Fa0/21`.

*Note:* *IEEE 802.1Q* is the commonly used standard, that defines the format of *VLAN-tagged* frames.

### *Spanning Tree Protocol (STP)* <a class="anchor" id="spanning-tree-protocol-stp"></a>

The *Spanning Tree Protocol* exists to allow for redundant links within a network topology, without looping frames being possible.

When a network is first powered up, *STP* negotiations begin, with every switch sending/receiving *STP*-specific messages, called *Bridge PDUs (BPDUs)*.

*STP* negotiations proceed as follows.

1. Electing the *Root Switch*.
    
    * Each switch has an assigned *Bridge ID (BID)*, consisting of a configurable 2-byte *priority*, followed by the 6-byte *MAC* address of the respective device.
    
    * Each *BPDU* sent contains information about the *BID* of the sender, and the *BID* of the sender's current elected root.
    
    * Whenever a *BPDU* is received with a *Root BID* lower than the receiver's current elected root, the receiver's current elected root is updated accordingly.

    * Over time, the switches converge on a single elected root, which becomes the *Root* Switch.

2. Choosing Each *Non-Root* Switch's *Root Port (RP)*.

    * Each *BPDU* also contains information about each sender's *cost* to reach the root. On the *Root Switch*, this cost is zero. On a directly connected switch, this cost is zero, plus the configured *adminstrative cost* of the physical interface connected to the root, which shall become its *Root Port*.

    * On a non-directly connected switch, this cost is calculated per physical interface, summing the *cost* in the *BPDU* received over that link, with the *adminstrative cost* of the respective physical interface. Upon comparison, the physical interface with the least cost becomes the *Root Port*.

3. Choosing the *Designated Port (DP)* on each link.

    * On each link, the *DP* is the physical interface with the lower cost to reach the root. Any non-*DP*s of a switch are shut off, except for the *RP*. 

    * Since on the *Root Switch*, all physical interfaces will become *DP*, the *Root Switch* always has all its physical interfaces up.

The figure below shows an example of a *LAN*, post *STP* negotiations.

<img src="../../.img/stp-topology.jpg" width="500" />

*Note:* If the topology changes, because *BDPU*s are sent periodically, switches are able to detect this change and re-negotiate, until the network converges again.